In [30]:
import pandas as pd

df = pd.read_csv('preprocessed_whole_dataset.csv')
df

,Unnamed: 0,선종,상태,state,vessel_type,nation_type
0,0,낚시어선,이동,1,8,1
1,1,낚시어선,조업,0,8,1
2,2,낚시어선,표류,2,8,1
3,3,등광조망,이동,1,2,0
4,4,등광조망,표류,2,2,0
...,...,...,...,...,...,...
1576,1576,범장망,이동,1,3,0
1577,1577,범장망,표류,2,3,0
1578,1578,유망,표류,2,1,0
1579,1579,타망,조업,0,0,0


In [31]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical


# 이미지 디렉토리 경로
image_dir = 'imgdata'  # 경로 수정 필요

# 이미지 읽기 및 전처리
images = []
for filename in os.listdir(image_dir):
    if filename.endswith(('.jpg', '.png')):  # 여러 확장자를 처리
        img_path = os.path.join(image_dir, filename)
        try:
            image = Image.open(img_path).convert('RGB').resize((256, 256))  # 이미지 크기 조정 및 RGB 변환
            image_array = img_to_array(image) / 255.0  # 정규화
            images.append(image_array)
        except Exception as e:
            print(f"Error loading image {filename}: {e}")

# 이미지 데이터를 NumPy 배열로 변환
images = np.array(images)

train_data = images
train_data
target = df['vessel_type']
target = to_categorical(target, num_classes=10)
target

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [32]:
import tensorflow as tf
from tensorflow.keras import layers, Model

def ResNet50(input_shape=(256, 256, 3), num_classes=10):
    inputs = tf.keras.Input(shape=input_shape)
    
    # MobileNetV2 backbone
    backbone = tf.keras.applications.ResNet50(input_shape=input_shape, include_top=False, weights='imagenet')(inputs)
    
    # Add custom head for classification
    x = layers.GlobalAveragePooling2D()(backbone)
    x = layers.Dense(128, activation='softmax')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs, name='ResNet50')
    return model

model = ResNet50()
model.summary()

Model: "ResNet50"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 8, 8, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,851,274 (90.99 MB)

 Trainable params: 23,798,154 (90.78 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [33]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [38]:
history = model.fit(train_data, target, epochs=150, verbose=1)

Epoch 1/150
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 84ms/step - accuracy: 0.4049 - loss: 1.8342
Epoch 2/150
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 84ms/step - accuracy: 0.3994 - loss: 1.8493
Epoch 3/150
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 84ms/step - accuracy: 0.4081 - loss: 1.8116
Epoch 4/150
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 85ms/step - accuracy: 0.3988 - loss: 1.8161
Epoch 5/150
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 84ms/step - accuracy: 0.3987 - loss: 1.8359
Epoch 6/150
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 84ms/step - accuracy: 0.4033 - loss: 1.8143
Epoch 7/150
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 83ms/step - accuracy: 0.4102 - loss: 1.7868
Epoch 8/150
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 84ms/step - accuracy: 0.4419 - loss: 1.7275
Epoch 9/150
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 84ms/step - accuracy: 0.4177 - loss: 1.7590
Epoch 10/150
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 84ms/step - accuracy: 0.4141 - loss: 1.7717
Epoch 11/150
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 83ms/step - accuracy: 0.4213 - loss: 1.7884
Epoch 12/150
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 84ms/step

In [44]:
# import os
# import numpy as np
# from PIL import Image
# from tensorflow.keras.preprocessing.image import img_to_array

# test_image_dir = '테스트셋'  # 이미지 폴더 경로

# # 이미지 읽기 및 전처리
# test_images = []  # 리스트로 선언
# for filename in os.listdir(test_image_dir):
#     if filename.lower().endswith(('.jpg', '.png')):  # 확장자를 소문자로 변환 후 비교
#         img_path = os.path.join(test_image_dir, filename)
#         try:
#             image = Image.open(img_path).convert('RGB').resize((256, 256))  # 이미지 크기 조정 및 RGB 변환
#             image_array = img_to_array(image) / 255.0  # 이미지를 배열로 변환하고 정규화
#             test_images.append(image_array)  # 리스트에 이미지 추가
#         except Exception as e:
#             print(f"Error loading image {filename}: {e}")

# # 이미지 데이터를 NumPy 배열로 변환
# test_images = np.array(test_images)  # 리스트를 한 번에 NumPy 배열로 변환

# test_data = test_images
# test_data

array([[[[0.7764706 , 0.75686276, 0.73333335],
         [0.3882353 , 0.36862746, 0.3529412 ],
         [0.26666668, 0.24705882, 0.24705882],
         ...,
         [0.26666668, 0.25882354, 0.26666668],
         [0.26666668, 0.25882354, 0.27058825],
         [0.27058825, 0.2627451 , 0.2784314 ]],

        [[0.7764706 , 0.7607843 , 0.7372549 ],
         [0.21568628, 0.20392157, 0.19215687],
         [0.        , 0.        , 0.        ],
         ...,
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.00392157]],

        [[0.7764706 , 0.7647059 , 0.7372549 ],
         [0.23137255, 0.21960784, 0.2       ],
         [0.01960784, 0.00392157, 0.00392157],
         ...,
         [0.00784314, 0.        , 0.00784314],
         [0.01176471, 0.00392157, 0.01568628],
         [0.01568628, 0.00784314, 0.02352941]],

        ...,

        [[0.76862746, 0.77254903, 0.7411765 ],
         [0.21568628, 0.21568628, 0.20392157]

In [45]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import img_to_array

test_image_dir = '테스트셋'  # 이미지 폴더 경로

# 이미지 읽기 및 전처리
test_images = []  # 리스트로 선언

# 파일 목록을 가져오고 숫자 순서대로 정렬
file_list = sorted(os.listdir(test_image_dir), key=lambda x: int(os.path.splitext(x)[0]))

for filename in file_list:
    if filename.lower().endswith(('.jpg', '.png')):  # 확장자를 소문자로 변환 후 비교
        img_path = os.path.join(test_image_dir, filename)
        try:
            image = Image.open(img_path).convert('RGB').resize((256, 256))  # 이미지 크기 조정 및 RGB 변환
            image_array = img_to_array(image) / 255.0  # 이미지를 배열로 변환하고 정규화
            test_images.append(image_array)  # 리스트에 이미지 추가
        except Exception as e:
            print(f"Error loading image {filename}: {e}")

# 이미지 데이터를 NumPy 배열로 변환
test_images = np.array(test_images)  # 리스트를 한 번에 NumPy 배열로 변환

test_data = test_images
print(test_data)


[[[[0.7764706  0.75686276 0.73333335]
   [0.3882353  0.36862746 0.3529412 ]
   [0.26666668 0.24705882 0.24705882]
   ...
   [0.26666668 0.25882354 0.26666668]
   [0.26666668 0.25882354 0.27058825]
   [0.27058825 0.2627451  0.2784314 ]]

  [[0.7764706  0.7607843  0.7372549 ]
   [0.21568628 0.20392157 0.19215687]
   [0.         0.         0.        ]
   ...
   [0.         0.         0.        ]
   [0.         0.         0.        ]
   [0.         0.         0.00392157]]

  [[0.7764706  0.7647059  0.7372549 ]
   [0.23137255 0.21960784 0.2       ]
   [0.01960784 0.00392157 0.00392157]
   ...
   [0.00784314 0.         0.00784314]
   [0.01176471 0.00392157 0.01568628]
   [0.01568628 0.00784314 0.02352941]]

  ...

  [[0.76862746 0.77254903 0.7411765 ]
   [0.21568628 0.21568628 0.20392157]
   [0.         0.         0.        ]
   ...
   [0.00784314 0.         0.01568628]
   [0.00784314 0.         0.02352941]
   [0.01176471 0.         0.02745098]]

  [[0.76862746 0.77254903 0.7411765 ]
   [0.2

In [46]:
predictions = model.predict(test_data)
# binary_predictions = (predictions > 0.5).astype(int)
class_predictions = np.argmax(predictions, axis=1)

print(class_predictions)
len(class_predictions)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
[7 6 8 6 3 2 0 2 3 3 2 3 3 0 2 0 8 3 8 8 1 2 0 3 3 7 3 3 3 3 2 1 3 0 3 3 2
 6 1 6 4 8 8 3 0 3 1 0 3 0 0 0 3]


53

In [37]:
test_history = model.evaluate(test_data)
test_loss, test_accuracy = test_history

print(f"Test loss: {test_loss:.8f}")
print(f"Test accuracy: {test_accuracy *  100.:.2f}")

ValueError: None values not supported.

In [ ]:
# classification report를 출력하려면 실제 레이블값 필요함
# from sklearn.metrics import classification_report

# report = classification_report(, class_predictions)
# print('Classification Report:')
# print(report)

ValueError: Found input variables with inconsistent numbers of samples: [1581, 53]